In [6]:
# installing missing dependencies
import warnings
warnings.filterwarnings('ignore')
!pip3 install sklearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [126]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
import pickle

In [127]:
df_yelp_business = pd.read_json('/Users/viprabha/Downloads/yelp_dataset/yelp_academic_dataset_business.json', lines=True)
df_yelp_business.fillna('NA', inplace=True)
# we want to make sure we only work with restaurants -- nothing else
df_yelp_business = df_yelp_business[df_yelp_business['categories'].str.contains('Restaurants')]
print('Final Shape: ',df_yelp_business.shape)

Final Shape:  (52268, 14)


In [128]:
df_yelp_review_iter = pd.read_json('/Users/viprabha/Downloads/yelp_dataset/yelp_academic_dataset_review.json', chunksize=100000, lines=True)

In [129]:
df_yelp_review = pd.DataFrame()

i=0
for df in df_yelp_review_iter:
    df = df[df['business_id'].isin(df_yelp_business['business_id'])]
    df_yelp_review = pd.concat([df_yelp_review, df])


In [130]:
print(df_yelp_review.shape)
df_yelp_review = df_yelp_review[:1000000]
print(df_yelp_review.shape)

(4724471, 9)
(1000000, 9)


In [131]:
df_yelp_business = df_yelp_business[df_yelp_business['business_id'].isin(df_yelp_review['business_id'])]

In [132]:
print('Final businesses shape: ', df_yelp_business.shape)
print('Final review shape: ', df_yelp_review.shape)

Final businesses shape:  (14204, 14)
Final review shape:  (1000000, 9)


In [133]:
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    return text

In [134]:
%%time
df_yelp_review['text'] = df_yelp_review['text'].apply(clean_text)

CPU times: user 4min 27s, sys: 38.7 s, total: 5min 6s
Wall time: 5min 9s


In [135]:
vectorizer_reviews = CountVectorizer(min_df = .01,max_df = .99, tokenizer = WordPunctTokenizer().tokenize)
vectorized_reviews = vectorizer_reviews.fit_transform(df_yelp_review['text'])

In [245]:
print(type(vectorizer_reviews))

<class 'sklearn.feature_extraction.text.CountVectorizer'>


In [137]:
' | '.join(vectorizer_reviews.get_feature_names()[:100]) # only the first 100

'! | + | - | 00 | 1 | 10 | 12 | 15 | 2 | 20 | 3 | 30 | 4 | 5 | 50 | 6 | 7 | 8 | : | ; | a | able | about | absolutely | accommodating | across | actually | add | added | addition | afternoon | again | ago | ahead | all | almost | along | already | also | although | always | am | amazing | ambiance | amount | and | another | anyone | anything | anyway | anywhere | appetizer | appetizers | are | area | around | arrived | as | ask | asked | ate | atmosphere | attention | attentive | authentic | available | average | avocado | away | awesome | awful | back | bacon | bad | baked | bar | barely | bartender | based | basically | bbq | be | beans | beautiful | beef | beer | beers | before | behind | believe | best | better | beyond | big | bill | birthday | bit | bite | black | bland'

In [289]:


# vectorizer_categories = CountVectorizer(min_df = 1, max_df = 1., tokenizer = lambda x: x.split(', '))
vectorizer_categories = CountVectorizer(min_df = 1, max_df = 1., tokenizer = WordPunctTokenizer().tokenize)

vectorized_categories = vectorizer_categories.fit_transform(df_yelp_business['categories'])

In [290]:
print(type(vectorizer_categories))

<class 'sklearn.feature_extraction.text.CountVectorizer'>


In [291]:
' | '.join(vectorizer_categories.get_feature_names()[10:110]) # only the first 100

'active | activities | adoption | adult | advertising | afghan | african | air | airport | amateur | american | ammo | amusement | animal | antiques | appliances | aquariums | arabic | arcades | arenas | argentine | armenian | art | arts | asian | assistance | australian | austrian | auto | automotive | bagels | bakeries | bangladeshi | banks | bar | barbeque | barbers | bars | bartenders | basque | bath | battery | batting | beaches | beauty | bed | beer | belgian | beverage | bike | bikes | billiards | bistros | blues | boat | boating | body | books | bookstore | bookstores | bowling | bowls | brasseries | brazilian | breakfast | breweries | brewpubs | british | brunch | bubble | buddhist | buffets | building | buildings | burgers | burmese | bus | business | butcher | cabaret | cafe | cafes | cafeteria | cages | cajun | cake | cakes | calabrian | cambodian | canadian | candy | cannabis | cantonese | car | cards | care | caribbean | casinos | caterers | cemeteries'

In [141]:
%%time
from scipy import sparse
businessxreview = sparse.csr_matrix(pd.get_dummies(df_yelp_review['business_id']).values)

CPU times: user 4min 10s, sys: 18.3 s, total: 4min 29s
Wall time: 4min 42s


In [176]:
sparse.save_npz("businessxreview.npz", businessxreview)
businessxreview_read = sparse.load_npz("businessxreview.npz")

In [292]:

# sparse.save_npz("vectorizer_reviews.npz", vectorizer_reviews)
# vectorizer_reviews_read = sparse.load_npz("vectorizer_reviews.npz")

# sparse.save_npz("vectorizer_categories.npz", vectorizer_categories)
# vectorizer_categories_read = sparse.load_npz("vectorizer_categories.npz")

with open('vectorizer_reviews.pkl', 'wb') as fout:
    pickle.dump(vectorizer_reviews, fout)
    

with open('vectorizer_reviews.pkl', 'rb') as f:
    vectorizer_reviews_read = pickle.load(f)
    
# import joblib
# joblib.dump(vectorizer_categories, 'vectorizer_categories.joblib')

# vectorizer_categories_read = joblib.load('vectorizer_categories.joblib')

with open('vectorizer_categories.pkl', 'wb') as fout:
    pickle.dump(vectorizer_categories,fout)
    

with open('vectorizer_categories.pkl', 'rb') as f:
    vectorizer_categories_read = pickle.load(f)

# vectorizer_reviews, vectorizer_categories
# vectorizer_reviews_read, vectorizer_categories_read
# Saving vectorized reviews 
with open('vectorizer.pkl', 'wb') as fout:
    pickle.dump((vectorized_reviews, vectorized_categories), fout)
    
# Saving vectorized categories
with open('vectorizer.pkl', 'rb') as f:
    vectorized_reviews_read, vectorized_categories_read = pickle.load(f)


In [241]:
print(type(vectorized_reviews), type(vectorized_reviews_read))

<class 'scipy.sparse._csr.csr_matrix'> <class 'scipy.sparse._csr.csr_matrix'>


In [242]:
# Business data pickle
df_yelp_business.to_pickle("df_yelp_business.pkl")
df_yelp_business_read = pd.read_pickle("df_yelp_business.pkl")
# Review data pickle
df_yelp_review.to_pickle("df_yelp_review.pkl")
df_yelp_review_read = pd.read_pickle("df_yelp_review.pkl")

In [293]:

print('restuarants x categories: \t', vectorized_categories_read.shape) 
print('restuarants x reviews: \t\t' , businessxreview_read.shape) 
print('reviews x words: \t\t', vectorized_reviews_read.shape)

restuarants x categories: 	 (14204, 648)
restuarants x reviews: 		 (1000000, 14204)
reviews x words: 		 (1000000, 894)


In [227]:
# to choose a restaurant, just copy the business id and paste it in the next cell
# you can always rerun the cell to choose another restuarant. 
df_yelp_business_read.sample(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
646,A1lAqN3SOaBLoo6NqVcy4Q,McDonald's,11110 Causeway Blvd,Brandon,FL,33511,27.925332,-82.325395,2.0,6,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Sandwiches, Burgers, Food, Fast Food, Restaura...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
41945,R3FDYMQBrMpkUquwE-eniQ,The Continental Restaurant and Martini Bar,138 Market St,Philadelphia,PA,19106,39.949693,-75.143623,4.0,638,0,"{'NoiseLevel': 'u'average'', 'BusinessAcceptsC...","Restaurants, American (New), Lounges, Nightlif...","{'Monday': '16:0-21:0', 'Tuesday': '16:0-21:0'..."
22259,tSmvpDRwqcXG2tXO9f61ZA,San Lucas Mexican Restaurant,2600 Federal St,Camden,NJ,08105,39.945942,-75.088741,3.5,38,1,"{'BikeParking': 'True', 'HasTV': 'True', 'Rest...","Restaurants, Mexican","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
4501,JszJI2Ewenggj8zh1o3B-Q,Peggs Restaurant On the Boulevard,1001 Crossroads Pl,High Ridge,MO,63049,38.464304,-90.527598,3.0,57,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsGoo...","Italian, Restaurants, Pizza","{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3..."
18778,O5LnTroDDouDbjdjufXJsg,El Agavero,"7657 Hwy 70 S, Ste 104",Nashville,TN,37221,36.077126,-86.951461,4.0,112,1,"{'RestaurantsPriceRange2': '1', 'Ambience': '{...","Mexican, Restaurants, American (New)","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3..."
25693,ZWh4bJk5Upc1MN4WH8Mm_g,Just Judy's,2210 E 54th St,Indianapolis,IN,46220,39.851989,-86.124365,3.0,5,0,"{'HasTV': 'True', 'Ambience': '{'romantic': Fa...","Steakhouses, Bars, Nightlife, Restaurants",NA
43859,XYwx1tsEB3_G0tgS6l-0PQ,Amuse,1421 Arch St,Philadelphia,PA,19102,39.954825,-75.164064,3.5,154,1,"{'RestaurantsTakeOut': 'True', 'Alcohol': 'u'f...","Food, Nightlife, French, Restaurants, Bars","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
33613,DIkXRqrDdW_86tSfdE1TWQ,Los Chilaquiles,559 Stewart's Ferry Pike,Nashville,TN,37214,36.158208,-86.635247,2.5,14,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Mexican, Restaurants","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'..."
25724,gwdREcEh7uXVVN4Vihgmlw,Joe Pesce,833 Haddon Ave,Collingswood,NJ,08108,39.915333,-75.065525,3.5,25,0,"{'RestaurantsReservations': 'True', 'Restauran...","Seafood, Restaurants","{'Tuesday': '17:0-22:0', 'Wednesday': '17:0-22..."
23596,2YowFoVtjOiZftrCldUyWA,Fit-Flavors,8566 Eager Rd,Saint Louis,MO,63144,38.628205,-90.340526,4.0,46,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Caterers, Event Planning & Services, Dietitian...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ..."


In [192]:
business_choose = 'hcL7e9njsG0AboZpEVF5ww' 

In [228]:
new_reviews = df_yelp_review_read.loc[df_yelp_review_read['business_id'] == business_choose, 'text']

In [229]:
print('\n'.join([r[:100] for r in new_reviews.tolist()])) # restaurant reviews

glad found place wings really good reasonable price recently delved calzone area menu real treat way
second time ordered delivery here first time delivery took hour half gave 50 next delivery order fir
ive ordered twice since close enough - i tried pizza instead ordered chicken parm sandwich grilled c
best pizza spot far found area not best philly def one best area prices really good 
offered coupon restaurant com place figured try since fairly close way home work worst ordered chick
got chicken cheesesteak fries soda to - go chicken cheesesteak loaded chunks fresh chicken well - se
place amazing offers good pizza amazing customer service owner generous person city definitely 5 5 
pizza places nearby definitely find better service quality price here called ahead ordered white ext
pretty nice place pizza stromboli soooo good probably one best area prices reasonable owner super fr
pizza shop around house pizzas good stromboli best thing menu 
love place lamb gyros good also get domenics 

In [230]:
new_categories = df_yelp_business_read.loc[df_yelp_business_read['business_id'] == business_choose, 'categories']

In [231]:
new_categories.tolist()

['Restaurants, Pizza']

In [295]:
from scipy.spatial.distance import cdist
# find most similar reviews
# dists1 = cdist(vectorizer_reviews.transform(new_reviews).todense().mean(axis=0), 
#               vectorized_reviews.T.dot(businessxreview).T.todense(), 
#                metric='correlation')
# # find most similar categories
# dists2 = cdist(vectorizer_categories.transform(new_categories).todense().mean(axis=0), 
#               vectorized_categories.todense(), 
#                metric='correlation')

dists1 = cdist(vectorizer_reviews_read.transform(new_reviews).todense().mean(axis=0), 
              vectorized_reviews.T.dot(businessxreview_read).T.todense(), 
               metric='correlation')
# find most similar categories
dists2 = cdist(vectorizer_categories_read.transform(new_categories).todense().mean(axis=0), 
              vectorized_categories_read.todense(), 
               metric='correlation')


In [296]:
# combine the two vectors in one matrix
dists_together = np.vstack([dists1.ravel(), dists2.ravel()]).T

In [297]:
dists_together

array([[0.51163994, 0.4364379 ],
       [0.47852485, 0.44078244],
       [0.52193259, 0.41699354],
       ...,
       [0.41436213, 0.41593917],
       [0.56367904, 0.36134078],
       [0.633277  , 0.33488372]])

In [298]:
# this is a key cell: how are we going to prioritize ?
dists = dists_together.mean(axis=1)

In [167]:
dists

array([0.59965378, 0.59711491, 0.62974539, ..., 0.59213069, 0.60673653,
       0.56758368])

In [168]:
# select the closest 10
closest = dists.argsort().ravel()[:10]

In [299]:
df_yelp_business_read.loc[df_yelp_business_read['business_id']== business_choose, ['business_id', 'categories', 'name', 'stars']]

,business_id,categories,name,stars
23133,hcL7e9njsG0AboZpEVF5ww,"Restaurants, Pizza",Domenic's Pizzeria,3.5


In [300]:
df_yelp_business_read.loc[df_yelp_business_read['business_id'].isin(df_yelp_business_read['business_id'].iloc[closest]), ['business_id', 'categories', 'name', 'stars']]

,business_id,categories,name,stars
830,clOOOBYvd1aUcFxA9uhFjQ,"Pizza, Restaurants",Tony Sopranos Pizza,3.0
15733,7Us4mR7VvyuOmDBIwQj6RQ,"Restaurants, Pizza",A B C Pizza House,3.5
22207,py2kSvhHg5B6Z9ZrLHRjuQ,"Pizza, Restaurants",Gino's Ristorante & Pizzeria,4.0
23928,YV-kXCOEnO3H5Lsj0rH2Hg,"Restaurants, Pizza",Franco's Pizza & Steak House,4.5
24833,QW1-KgjRlJWjr8uh9Y78gw,"Pizza, Restaurants",The Original Tony's Pizza,3.5
26033,KjJDvCg5wuK-wyRy9OYJYQ,"Restaurants, Pizza",Bally Pizza,3.0
32878,I_1GURBE73oKkPHNCvCCiQ,"Pizza, Restaurants",Pizza Mania,4.0
34330,jukJ94bXlgu4KU4x8xANpg,"Restaurants, Pizza",Apollo Pizza,2.0
38424,ZZASTnu-8vFjHkvGgatUSA,"Restaurants, Pizza",Sciarrino's Pizzeria,2.0
38486,9oZc6qwbJfO0SuKXEMh27g,"Pizza, Restaurants",Pizza Roma,3.0


In [2]:
# imports 
import pandas as pd
import pickle
from scipy import sparse
from scipy.spatial.distance import cdist
import numpy as np
import json


In [9]:
# integration code

# loading business review sparse matrix
businessxreview_read = sparse.load_npz("/Users/viprabha/Music/rec_files/businessxreview.npz")

# loading countvectorized reviews
with open('/Users/viprabha/Music/rec_files/vectorizer_reviews.pkl', 'rb') as f:
    vectorizer_reviews_read = pickle.load(f)
    
# loading countvectorized categories
with open('/Users/viprabha/Music/rec_files/vectorizer_categories.pkl', 'rb') as f:
    vectorizer_categories_read = pickle.load(f)
    
    
# loading transformed reviews and categories
with open('/Users/viprabha/Music/rec_files/vectorizer.pkl', 'rb') as f:
#     vectorized_reviews_read, vectorized_categories_read = pickle.load(f)
    vectorized_reviews_read, vectorized_categories_read = pickle.load(f)
    
# business data
df_yelp_business_read = pd.read_pickle("/Users/viprabha/Music/rec_files/df_yelp_business.pkl")

# review data 
df_yelp_review_read = pd.read_pickle("/Users/viprabha/Music/rec_files/df_yelp_review.pkl")


In [10]:
business_choose = 'hcL7e9njsG0AboZpEVF5ww' 

new_reviews = df_yelp_review_read.loc[df_yelp_review_read['business_id'] == business_choose, 'text']

print('\n'.join([r[:100] for r in new_reviews.tolist()])) # restaurant reviews


glad found place wings really good reasonable price recently delved calzone area menu real treat way
second time ordered delivery here first time delivery took hour half gave 50 next delivery order fir
ive ordered twice since close enough - i tried pizza instead ordered chicken parm sandwich grilled c
best pizza spot far found area not best philly def one best area prices really good 
offered coupon restaurant com place figured try since fairly close way home work worst ordered chick
got chicken cheesesteak fries soda to - go chicken cheesesteak loaded chunks fresh chicken well - se
place amazing offers good pizza amazing customer service owner generous person city definitely 5 5 
pizza places nearby definitely find better service quality price here called ahead ordered white ext
pretty nice place pizza stromboli soooo good probably one best area prices reasonable owner super fr
pizza shop around house pizzas good stromboli best thing menu 
love place lamb gyros good also get domenics 

In [13]:
new_categories = df_yelp_business_read.loc[df_yelp_business_read['business_id'] == business_choose, 'categories']

print(new_categories.tolist())
dists1 = cdist(vectorizer_reviews_read.transform(new_reviews).todense().mean(axis=0), 
              vectorized_reviews_read.T.dot(businessxreview_read).T.todense(), 
               metric='correlation')
# find most similar categories
dists2 = cdist(vectorizer_categories_read.transform(new_categories).todense().mean(axis=0), 
              vectorized_categories_read.todense(), 
               metric='correlation')


['Restaurants, Pizza']


In [16]:
# combine the two vectors in one matrix
dists_together = np.vstack([dists1.ravel(), dists2.ravel()]).T
# this is a key cell: how are we going to prioritize ?
dists = dists_together.mean(axis=1)

In [18]:
# select the closest 10
closest = dists.argsort().ravel()[:10]

df_yelp_business_read.loc[df_yelp_business_read['business_id']== business_choose, ['business_id', 'categories', 'name', 'stars']]

recommendations = df_yelp_business_read.loc[df_yelp_business_read['business_id'].isin(df_yelp_business_read['business_id'].iloc[closest]), ['business_id', 'categories', 'name', 'stars']]

In [44]:
result = recommendations.to_json(orient = 'index')
result = json.loads(result)
for ele in result:
    print(result[ele])

{'business_id': 'clOOOBYvd1aUcFxA9uhFjQ', 'categories': 'Pizza, Restaurants', 'name': 'Tony Sopranos Pizza', 'stars': 3.0}
{'business_id': '7Us4mR7VvyuOmDBIwQj6RQ', 'categories': 'Restaurants, Pizza', 'name': 'A B C Pizza House', 'stars': 3.5}
{'business_id': 'py2kSvhHg5B6Z9ZrLHRjuQ', 'categories': 'Pizza, Restaurants', 'name': "Gino's Ristorante & Pizzeria", 'stars': 4.0}
{'business_id': 'YV-kXCOEnO3H5Lsj0rH2Hg', 'categories': 'Restaurants, Pizza', 'name': "Franco's Pizza & Steak House", 'stars': 4.5}
{'business_id': 'QW1-KgjRlJWjr8uh9Y78gw', 'categories': 'Pizza, Restaurants', 'name': "The Original Tony's Pizza", 'stars': 3.5}
{'business_id': 'KjJDvCg5wuK-wyRy9OYJYQ', 'categories': 'Restaurants, Pizza', 'name': 'Bally Pizza', 'stars': 3.0}
{'business_id': 'I_1GURBE73oKkPHNCvCCiQ', 'categories': 'Pizza, Restaurants', 'name': 'Pizza Mania', 'stars': 4.0}
{'business_id': 'jukJ94bXlgu4KU4x8xANpg', 'categories': 'Restaurants, Pizza', 'name': 'Apollo Pizza', 'stars': 2.0}
{'business_id': '

In [5]:
pip list

Package             Version
------------------- -----------
aiohttp             3.7.4.post0
appdirs             1.4.4
appnope             0.1.2
argon2-cffi         21.1.0
async-timeout       3.0.1
attrs               21.2.0
backcall            0.2.0
bleach              4.1.0
certifi             2021.5.30
cffi                1.14.6
chardet             4.0.0
charset-normalizer  2.0.6
click               8.1.2
cycler              0.11.0
debugpy             1.4.3
decorator           5.1.0
defusedxml          0.7.1
dill                0.3.4
distlib             0.3.2
entrypoints         0.3
filelock            3.0.12
fonttools           4.32.0
idna                3.2
ipykernel           6.4.1
ipython             7.28.0
ipython-genutils    0.2.0
ipywidgets          7.6.5
iso8601             0.1.16
jedi                0.18.0
Jinja2              3.0.1
joblib              1.1.0
jsonschema          3.2.0
jupyter             1.0.0
jupyter-client      7.0.3
jupyter-console     6.4.0
jupyter-core   